# Download the kaggle data set.

In [1]:

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c humpback-whale-identification
!rm -rf input
!mkdir -p input
!unzip -q train.zip -d input/train
#!ls input/train

kaggle.json
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split


from keras import layers
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)


Using TensorFlow backend.


# Explore the training data.
Let us explore the training data.


In [3]:
train_df = pd.read_csv("train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [4]:
train_df.Id.value_counts()

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
w_700ebb4      57
w_3de579a      54
w_564a34b      51
w_fd3e556      50
w_88e4537      49
w_2b069ba      48
w_d405854      47
w_789c969      45
w_f0fe284      45
w_343f088      40
w_778e474      40
w_5e8e218      40
w_60ce6fc      37
w_a9304b9      37
w_5a2634c      37
w_6822dbc      36
w_af367c3      35
w_1ca9ab1      34
w_f765256      34
w_17b0d3a      33
w_d72771c      32
w_8c25681      31
w_08630fd      31
w_6cda039      31
w_04003e9      30
             ... 
w_90df70c       1
w_ba668da       1
w_2e1416e       1
w_1e81b43       1
w_702b2c4       1
w_74192ac       1
w_1ee38d5       1
w_34941d8       1
w_56e52fb       1
w_2b1b04e       1
w_e759cd6       1
w_f0ba4a2       1
w_d4fc12d       1
w_2ae40f9       1
w_bc9188e       1
w_f347577       1
w_97074d8       1
w_408f9ea       1
w_b51e6f6       1
w_46a38a6       1
w_3c58b68       1
w_01ed442       1
w_7d9d014       1
w_ae70500       1
w_8261f4d 

In [5]:
print('Number of rows in train.csv', len(train_df))

Number of rows in train.csv 25361


# Identify the data points

## train.csv
There are 25361 rows in train.csv.  Which corresponds to the image entries in train.zip
We can see that the train.csv file has two data fields.  
* Image : The whale image file name
* Id is the whale Id.
Each whale is assigned a unique Id.  The unidentified whale's are assigned an Id new_whale.  


## train.zip
There are 25361 image files in train.zip file.  It has been extracted to input/train folder.  The filename corresponds to the Image column in train.csv file.

# Split the data into training, validation & test datasets





In [0]:
# Encode labels to integers using sklearning.preprocessing.LabelEncoder
# Convert the integer encoded array to 
le = LabelEncoder()
def transform_category(y):
  le.fit(y)
  y_transform = le.fit_transform(y)
  y_transform = np_utils.to_categorical(y_transform, num_classes=len(le.classes_))
  return y_transform

In [7]:
labels = train_df.Id
y_transform = transform_category(labels)

print(y_transform,len(y_transform))

X_train, X_tmp, Y_train, Y_tmp = train_test_split(train_df, y_transform, test_size=0.2, random_state=5)

X_val, X_test, Y_val, Y_test   = train_test_split(X_tmp, Y_tmp, test_size=0.5, random_state=5)

print('Training, Validation & testing data size', len(X_train),len(X_val), len(X_test))


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] 25361
Training, Validation & testing data size 20288 2536 2537


In [8]:
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

model.add(BatchNormalization(axis = 3, name = 'bn0'))
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), name='max_pool'))
model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool'))

model.add(Flatten())
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.8))
model.add(Dense(5005, activation='softmax', name='sm'))

print(model.output_shape)
model.summary()


W0718 03:16:40.223158 140048880281472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 03:16:40.243598 140048880281472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 03:16:40.257653 140048880281472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 03:16:40.289767 140048880281472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0718 03:16:40.290764 1400488802

(None, 5005)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 94, 94, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 94, 94, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 94, 94, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 47, 47, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 45, 45, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 45, 45, 64)        0         
_________________________________________________________________
avg_pool (AveragePooling2D)  (None, 15, 15, 64)        0       

In [0]:
def prepare_images(data):
    
    print("Preparing images")
    
    images = np.zeros((len(data), 100, 100, 3))
    
    count = 0
    
    for fig in data.Image:
        #load images into images of size 100x100x3
        img = image.load_img("input/train/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        images[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    count = 0
    
    print("Finished!")
            
    return images

In [10]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

W0718 03:16:42.237782 140048880281472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [11]:
x_train_images = prepare_images(X_train)
x_train_images /= 255

print("Shape X-train: ", x_train_images.shape)

x_val_images = prepare_images(X_val)
x_val_images /= 255

print("Shape X-val: ", x_val_images.shape)

x_test_images = prepare_images(X_test)
x_test_images /= 255

print("Shape X-test: ", x_test_images.shape)

Preparing images
Processing image:  1 ,  5e2572252.jpg
Processing image:  501 ,  b728ef1e9.jpg
Processing image:  1001 ,  942ab5de3.jpg
Processing image:  1501 ,  dd4cfa29f.jpg
Processing image:  2001 ,  614f10ee7.jpg
Processing image:  2501 ,  db9667359.jpg
Processing image:  3001 ,  86c9aa515.jpg
Processing image:  3501 ,  7f3aafbd2.jpg
Processing image:  4001 ,  6f0c3deb4.jpg
Processing image:  4501 ,  444b09aca.jpg
Processing image:  5001 ,  f532c9318.jpg
Processing image:  5501 ,  f2d3d0d0f.jpg
Processing image:  6001 ,  6ca37fe7c.jpg
Processing image:  6501 ,  3394e12db.jpg
Processing image:  7001 ,  feddb3aa9.jpg
Processing image:  7501 ,  3a8173905.jpg
Processing image:  8001 ,  16ddf58df.jpg
Processing image:  8501 ,  64b519010.jpg
Processing image:  9001 ,  c2a02f80e.jpg
Processing image:  9501 ,  770cb755e.jpg
Processing image:  10001 ,  803515118.jpg
Processing image:  10501 ,  5e8632b10.jpg
Processing image:  11001 ,  5f37d323c.jpg
Processing image:  11501 ,  204823b38.jpg

In [0]:
def map_per_image(label, predictions):
    """Computes the precision score of one image.

    Parameters
    ----------
    label : string
            The true label of the image
    predictions : list
            A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """    
    try:
        return 1 / (predictions[:5].index(label) + 1)
    except ValueError:
        return 0.0

def map_per_set(labels, predictions):
    """Computes the average over multiple images.

    Parameters
    ----------
    labels : list
             A list of the true labels. (Only one true label per images allowed!)
    predictions : list of list
             A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """
    return np.mean([map_per_image(l, p) for l,p in zip(labels, predictions)])

In [0]:
import gc
history = model.fit(x_train_images, Y_train, epochs=100, batch_size=100, verbose=1,
                   validation_data=(x_val_images,Y_val))
gc.collect()

W0718 03:20:48.420294 140048880281472 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 20288 samples, validate on 2536 samples
Epoch 1/100
20288/20288 [==============================] - 18s 908us/step - loss: 6.1984 - acc: 0.3798 - val_loss: 6.8944 - val_acc: 0.3825
Epoch 2/100
20288/20288 [==============================] - 13s 646us/step - loss: 5.9181 - acc: 0.3815 - val_loss: 5.9683 - val_acc: 0.3825
Epoch 3/100
20288/20288 [==============================] - 13s 646us/step - loss: 5.7978 - acc: 0.3815 - val_loss: 5.7611 - val_acc: 0.3825
Epoch 4/100
20288/20288 [==============================] - 13s 643us/step - loss: 5.7167 - acc: 0.3815 - val_loss: 5.7281 - val_acc: 0.3825
Epoch 5/100
20288/20288 [==============================] - 13s 637us/step - loss: 5.6655 - acc: 0.3815 - val_loss: 5.7618 - val_acc: 0.3825
Epoch 6/100
20288/20288 [==============================] - 13s 639us/step - loss: 5.6244 - acc: 0.3815 - val_loss: 5.7577 - val_acc: 0.3825
Epoch 7/100
20288/20288 [==============================] - 13s 641us/step - loss: 5.5912 - acc: 0.3815 - val_lo

In [0]:
pred = model.predict(x_test_images, verbose=1)
print(pred.shape)

In [0]:
predictions=[]
for i, p in enumerate(pred):
  predictions.append(le.inverse_transform(p.argsort()[-5:][::-1]).tolist())
print(map_per_set(X_test.Id, predictions ))